In [2]:
import httpx
import requests
import pandas as pd
import numpy as np
import re
import lxml.html as lx
import time
from bs4 import BeautifulSoup
import requests_cache
requests_cache.install_cache("fp")
from tqdm import tqdm
import string
import nltk
import warnings
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
all_job_urls = pd.read_csv("all_job_urls.csv")

In [30]:
HEADERS = {
    "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    "Accept-Encoding": "gzip",
    "Accept": "text/html",
    "Accept-Language": "en",
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Cookie": 'CTK=1hiq7b982kp0r800; CSRF=UI0smzR2tqo8tJ24OuSOj508sm9vHI8F; INDEED_CSRF_TOKEN=M2rWlP53k5jQfzlAKOPZPEyndcH9SQPw; hpnode=1; _ga=GA1.2.1145195577.1710268026; SURF=ItSgDBEj0VPErn3ajGry1QSo4GuVxYCF; _gcl_au=1.1.731099277.1710268048; LOCALE=en; MICRO_CONTENT_CSRF_TOKEN=QCX1xyVCzusGcsmtf9QmtsqKkp9O3XK1; PPID=""; cp=1; SHARED_INDEED_CSRF_TOKEN=M2rWlP53k5jQfzlAKOPZPEyndcH9SQPw; CMP_VISITED=1; cmppmeta="eNoBSAC3/8b/XBs7B0rSHIGEFJ46L9UdKTYVM6+sJmiVYxvpWk35oUR7UvfyJu9WAEje3S0/RCbbefeqNX5i+p2s6IlHhBtxVaU/Bn5K8Xj4IOE="; indeed_rcc="cmppmeta:LV:CTK"; cmp-jobsCta=hidden; _gid=GA1.2.335988752.1710918658; cmpsmeta="eNoBOADH/9aA1wR7fF6jCVoww2nY2biOLpn6KhfawGvqoP8GjMMSOM3mZ4mTYM26MD9JVM165pyD5+X6lXJnTBseuA=="; RJAS=v7f5cdfb8f61922b2:ie77c8615daba01c7; LC="co=US&hl=en_US"; ROJC=f54882cebf10ebd4:87375e52799bef99:d889eeefdb0ec0c3:1a2903e5ba3a600f:de4d586b8c555828:c4a0c5f43b1be38b:831587431a365711:e90b3572f6c08240:4484d7039ddfc55e:667062de587332f3; RSJC=a3084fb41e49fce0:44984acf92d08d60:e77c8615daba01c7:6e3f8d65e9c47a5f:a54d06e7f22b8fe2:7872f8291b2ac864:fb6af8fe9094d287:7f5cdfb8f61922b2:1a76a5f8066be394:bb2a9274c7891c7c; cf_clearance=ay8veu1OGi7SD5SYVhVOtIdmnhSBLf17DDMnj6Sdhz0-1710954247-1.0.1.1-CqvsQX2tXBrtamOVPXdMqLR8loGAPIarbzAYYGI_hNpiJlwugybckNkP.BPJhvJSL8Xp3f9mVm3TdC8S5lLkSw; __cf_bm=dY2GcM0MUBFHvUjJhJbAEkZG6s_y8VYHVoSAgK5MtQY-1710959648-1.0.1.1-vM6BVbtLlVrEwfVQzT353iaDKFrwx1Fzav0ly7zSVayTC4MehxtgBVhYpr1aFM2wvws6ZDOS40gaBOzYfi9fXw; _cfuvid=260GWXYbRFSZkaeqo1q4bJgEfdt7o04bZZjS1YPOMu4-1710959648791-0.0.1.1-604800000; LV="LA=1710959912:LV=1710932098:CV=1710959912:TS=1710268024"; RQ="q=data+scientist&l=&ts=1710959912294&pts=1710932177839:q=biostatistician&l=&ts=1710741322360:q=data+science+intern&l=&ts=1710646303290&pts=1710291574347:q=data+analyst+internship&l=&ts=1710291500827:q=data+analyst&l=&ts=1710291471839:q=data&l=&ts=1710291360613&pts=1710269684404"; JSESSIONID=48462AC846AB222A8F94D443A4F025CE; _gat=1'
}

In [9]:
all_job_urls = [item for sublist in all_job_urls for item in sublist]
all_job_urls[:10]

['https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dv5y2Tz_XTKJ5_FAkW3zrjLJOd2TmyCZSSGZAIb9qsK7ED9MMzJfjSU2w0kpoNwcJaml4oafevb-o3Ugsk_frYlGVEBMoPkqxp8whNJr5tOFQHG8bEFY2k5826KYL7Qe7WxLcxFxPzSStkzPo-54WgjEtX6iqpDHCeZe6BTK98nuTZ1jxzJ2mU2lB8SyaRwDTE6EBONmflvwxMysPYwZndWb0ouQJKfODLFgYCt6GvRsdFQFlvg_XOkikHU9NuLxlTWgbBOMKMcszHRVmfPXxOKyM_2IoTwjVT1oQHH9lDo_Qn75WrHrnqkVuzaP3SDZMuAi2zzwrYR5yPPfDpw6ZvNiL4nxzBSUaH4caQ2huYOjEXRX5aFt9XTZ_eqipagKZdw37PJBiypEEznj_kzUFkzWjwU6m3g8uOP148F--OVGcqaidp7urOgl3ppXDRmUTeVR7qGG57WbGgKSRw3znPtjEicSrkuh4WO67MpyYMHuXvqbUAdBMp8xpzPwlmlNVJpyZh9f-wJQJV_gXS-gAaMAK14p8BChu_uB7jYKjbBhWXTaqnW7Kp3V2dKz9YjHwp6SpTJl2sxXlYXwfRcz9x&xkcb=SoDm6_M3DlT6q7Tenh0LbzkdCdPP&camk=4HOcmqOLYrCEe3bkuslzNA==&p=0&fvj=0&vjs=3',
 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CnM4TERr6XuVBW3VTRACvDXDx9S3Pbn6a0SwhkmoLTiKD4BGD0VT7S6S0mhSg0W4vN4oo9e5UeuVScx57ooOWwlk-R3y6npUah7ze3WgfL68m5TudUhYdSk7h9Az2dKYTdKHCisLtjYcDGthvCmzluHdSrfIwNFzRLd6NDiH-vFORi6FRS0ZShmjxf00uHR4iyJD059HnP_OW2AYgug

In [11]:
all_job_soup = []
for url in tqdm(all_job_urls[:10], desc="Fetching job details", unit="job"):
    time.sleep(0.1)
    response = requests.get(url, headers=HEADERS, cookies={})
    if response.status_code == 200:
        all_job_soup.append(response.text)
print("All job details fetched!")

Fetching job details:   0%|          | 0/10 [00:00<?, ?job/s]

Fetching job details: 100%|██████████| 10/10 [00:02<00:00,  3.40job/s]

All job details fetched!


In [29]:
proxies = {'http': 'http://10.10.1.10:3128'}
response2 = requests.get(all_job_urls[0], headers=HEADERS, proxies=proxies)
print(response2)
response2.text

<Response [403]>


'<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .big-button,body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url()}body #challenge-error-tex

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# simplyhired

In [32]:
driver_path1 = " /Users/masheng/Library/Python/3.11/lib/python/site-packages"

options = webdriver.ChromeOptions()
options.add_argument(driver_path1)

driver1 = webdriver.Chrome(options=options)

In [61]:
driver1.get('https://www.simplyhired.com/search?q=data+scientist&l=')

In [73]:
all_job_details = pd.DataFrame(columns=['job_title','benifits','qualifications','full job description'])

In [97]:
next_job_card = driver1.find_element(By.CSS_SELECTOR, "[class='chakra-button css-1djbb1k']")

In [71]:
num = 0
page = 0

In [93]:
while True:
    while next_job_card != None:
        job_card = next_job_card
        job_card.click()
        time.sleep(3)

        # try:
        #     useless_window = driver1.find_element(By.CLASS_NAME,'modal_main gdGrid css-1gy9wmw e8e8plt4')
        #     close = useless_window.find_element(By.CLASS_NAME,'SVGInline modal_closeIcon')
        #     click = close.click()
        #     time.sleep(2)
        # except:
        #     pass

        job_details = pd.DataFrame({
            'job_title':None,'benifits':None,'qualifications':None,
            'full job description':None}, index=[0])

        try:
            job_title = driver1.find_element(By.CSS_SELECTOR, "[class = 'chakra-stack css-1iblfv6']")
            job_title = job_title.text
            print(job_title)
            job_details['job_title'] = job_title
        except:
            pass
        
        try:
            benifites = driver1.find_element(By.CSS_SELECTOR, "[data-testid = 'viewJobBodyJobBenefits']")
            benifites = benifites.text.split("\n")
            print(benifites)
            job_details['benifits'] = benifites
        except:
            pass


        try:
            qualifications = driver1.find_element(By.CSS_SELECTOR, "[data-testid = 'viewJobQualificationsContainer']")
            qualifications = qualifications.text.split("\n")
            print(qualifications)
            job_details['qualifications'] = qualifications
        except:
            pass

        try:
            full_job_description = driver1.find_element(By.CSS_SELECTOR, "[data-testid = 'viewJobBodyJobFullDescriptionContent']")
            full_job_description = full_job_description.text
            job_details['full job description'] = full_job_description
        except:
            pass


        all_job_details = pd.concat([all_job_details,job_details],ignore_index=True)
        num+=1

        try:
            next_job_card = job_card.find_element(By.CSS_SELECTOR, "[class='chakra-button css-1djbb1k']")
        except:
            next_job_card = None
    
    print(f"Page {page} - Total {num} jobs fetched!")
    
    try:
        next_button = driver1.find_element(By.CSS_SELECTOR, "[aria-label = 'Next page']")
        next_button.click()
        time.sleep(5)
        page+=1
    except:
        break

    # try:
    #     useless_window = driver1.find_element(By.CLASS_NAME,'modal_main gdGrid css-1gy9wmw e8e8plt4')
    #     close = useless_window.find_element(By.CLASS_NAME,'SVGInline modal_closeIcon')
    #     click = close.click()
    #     time.sleep(2)
    # except:
    #     pass

    next_job_card = driver1.find_element(By.CSS_SELECTOR, "[data-testid='searchSerpJob']")

all_job_details

Manager, Model Validation
['Paid time off', 'Volunteer time off']
['Qualifications', 'Financial modeling', '5 years', 'R', 'Mathematics', 'Finance', "Bachelor's degree", 'Risk management', 'Quantitative analysis', 'SaaS', 'RMF', 'Communication skills', 'Python']
Page 3 - Total 4 jobs fetched!
Sr. Data Analyst Contractor, NextGen
['Qualifications', 'Microsoft Excel', 'Merchandising', 'Supply chain', 'Process improvement', '3 years', 'SQL', 'Continuous improvement', 'Product management', 'Grocery store', 'Translation', 'Communication skills', 'Python', 'Analytics']
Sr. Data Analyst Contractor, NextGen
['Qualifications', 'Microsoft Excel', 'Merchandising', 'Supply chain', 'Process improvement', '3 years', 'SQL', 'Continuous improvement', 'Product management', 'Grocery store', 'Translation', 'Communication skills', 'Python', 'Analytics']
Page 4 - Total 6 jobs fetched!
Data Scientist
['Qualifications', 'TensorFlow', 'Research laboratory experience', 'Computer science', '7 years', 'Computer 

KeyboardInterrupt: 

In [101]:
# job_cards = driver1.find_elements(By.CSS_SELECTOR, "[data-testid='searchSerpJob']")
job_urls = [i.find_element(By.TAG_NAME,'a').get_attribute('href') for i in job_card]

In [105]:
all_job_urls = []

In [106]:
i=0
while True:
    job_card = driver1.find_elements(By.CSS_SELECTOR, "[data-testid='searchSerpJob']")
    job_urls = [i.find_element(By.TAG_NAME,'a').get_attribute('href') for i in job_card]
    all_job_urls = all_job_urls + job_urls
    print(f"Page {i} : {len(all_job_urls)} - Total {len(all_job_urls)} jobs fetched!")
    i+=1

    try:
        next_button = driver1.find_element(By.CSS_SELECTOR, "[aria-label = 'Next page']")
        next_button.click()
        time.sleep(3)
    except:
        break

Page 0 : 20 - Total 20 jobs fetched!
Page 1 : 40 - Total 40 jobs fetched!
Page 2 : 60 - Total 60 jobs fetched!
Page 3 : 80 - Total 80 jobs fetched!
Page 4 : 100 - Total 100 jobs fetched!
Page 5 : 120 - Total 120 jobs fetched!
Page 6 : 140 - Total 140 jobs fetched!
Page 7 : 160 - Total 160 jobs fetched!
Page 8 : 180 - Total 180 jobs fetched!
Page 9 : 200 - Total 200 jobs fetched!
Page 10 : 220 - Total 220 jobs fetched!
Page 11 : 240 - Total 240 jobs fetched!
Page 12 : 260 - Total 260 jobs fetched!
Page 13 : 280 - Total 280 jobs fetched!
Page 14 : 300 - Total 300 jobs fetched!
Page 15 : 320 - Total 320 jobs fetched!
Page 16 : 340 - Total 340 jobs fetched!
Page 17 : 360 - Total 360 jobs fetched!
Page 18 : 380 - Total 380 jobs fetched!
Page 19 : 400 - Total 400 jobs fetched!
Page 20 : 420 - Total 420 jobs fetched!
Page 21 : 440 - Total 440 jobs fetched!
Page 22 : 460 - Total 460 jobs fetched!
Page 23 : 480 - Total 480 jobs fetched!
Page 24 : 500 - Total 500 jobs fetched!
Page 25 : 520 - To

In [107]:
all_job_urls[:5]

['https://www.simplyhired.com/job/zRCtbfE8nzAXnn0021Xjd6GX3N9PWZt8nzAXlXGWsORQXVlYTSgheg?isp=1&jobCardTrackingKey=5-pdx1-0-1hpf7auakk57n800-91f9074d302543e2---6NYlbfkN0AlIVd5Z9xE1cdy5gk3DQPaPGBF7szVrnQGVwgGCyHZJ5O6fv2zoKFloNxUlDJYmO5CyIEnfcivASRpRU0m5npnZyGjZIKPECOIRc65YX_1J-F-qBFqOuR7MoVRj9b0b6jN91Vv9v3GRBTxF3HtsJJ76Cn5YVuyPHXVI6Mra6Uc-z8_HzVaQJJrlB-f6CVcSpDMaSIrpItRKjNp1J7jHp1hywoBexGKOi4t5A6qlVTUGREgmcUt4niYW3RUlWPZ-fdvf-0GmGPzIJ82Me5fZhynIfj3pQGHkU3GpxYmEdM-7C67xUlmoF-bVr1_nYFnEd9IpCEqRT2zho7OUSaJUjJfoKrtL5vez5rvYsa1qdYErGrKJLR7XCThK_sbe8xF0r6qpVhucv7Rv62FlmKir2qjCcOxmc62ZRVKA64xaQqTnR5bFcN4xGFa5HICErCI0Jbo2-KcaMiR5api-m0iTuca09ECYeddXH1fG1erXAH9O56Xuqu09Hb28_O-8QeLn9SuHWrYS1yauLR7NWEAC2VzdXNJeRa4LVuwjXE64rlHe7reJubaqy7w9fmdiukyiBcmY213UfsNSP0Vy_Yb59bcup9eXkKupluGg6KFigwgf4ewBK6Smea87MkHwkUFT6_BdDqNHCe9l_-TICoVtTF-s6V_0K0RCyQkj2SbEE0pKN71ROgWQTYu2fCsdgNc2OTjtoYube9n8sVsKM56FdXfMeMdALiSTh4%3D&xkcb=SoDF6_M3DvGkFc2fwR0LbzkdCdPP&camk=H-lBaXMUocJ7RJ1MQM28-g%3D%3D',
 'https://www.simplyh

In [151]:
pd.DataFrame(all_job_urls).to_csv("all_simplyhired_urls.csv", index=False)

In [110]:
HEADERS = {
    "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    "Accept-Encoding": "gzip",
    "Accept": "text/html",
    "Accept-Language": "en",
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Cookie": 'shk=1hpej3gtsmkkm801; csrf=CoA78HzgzcFPgttGNZvRzeo_poNSlsETD8MeHvzk-DbdqUWA05JfILc3CC404mI8hu453pOdbLSI81M_LNlZ; _ga=GA1.1.1189312297.1710960528; rq=%5B%22q%3Ddata%2Bscientist%26l%3D%26ts%3D1710972560480%22%5D; __cf_bm=aN_K675TYklod50qxgAAOmQdSQ_7ICBzeOqdWIT.2S0-1710983723-1.0.1.1-IRkjgI3bB9lZNXzFJki1qVhF.mCt3a6p.UIIUq.j02XTQBKySe_KNHPu2onGnuiE23UrES17yJE9I9Kj08nfmg; _ga_9GC5K2RCSP=GS1.1.1710984354.4.0.1710984354.0.0.0; OptanonConsent=isGpcEnabled=0&datestamp=Wed+Mar+20+2024+18%3A25%3A54+GMT-0700+(%E5%8C%97%E7%BE%8E%E5%A4%AA%E5%B9%B3%E6%B4%8B%E5%A4%8F%E4%BB%A4%E6%97%B6%E9%97%B4)&version=202308.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=b14705d6-2bab-4a15-9ca9-885fdfdc93c2&interactionCount=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1%2CC0007%3A0&AwaitingReconsent=false'
}
response = requests.get(all_job_urls[0], headers=HEADERS)
response

<Response [200]>

In [113]:
job_soup = BeautifulSoup(response.text, "html.parser")

In [121]:
benifites = job_soup.find_all('span',{'data-testid':'viewJobBenefitItem'})
benifites = [i.text for i in benifites]
print(benifites)

['Paid time off', 'Volunteer time off']


In [122]:
qualifications = job_soup.find_all('span',{'data-testid':'viewJobQualificationItem'})
qualifications = [i.text for i in qualifications]
print(qualifications)

['Financial modeling', '5 years', 'R', 'Mathematics', 'Finance', "Bachelor's degree", 'Risk management', 'Quantitative analysis', 'SaaS', 'RMF', 'Communication skills', 'Python']


In [125]:
full_job_description = job_soup.find('div',{'data-testid':'viewJobBodyJobFullDescriptionContent'})
full_job_description = full_job_description.text
print(full_job_description)

When you join Sallie Mae, you become a champion for all students.
We’re on a mission to power confidence as students begin their unique journey. To help them plan their higher education, successfully finish, and prepare for life after school. To help them Start smart. Learn big.
Students need guidance navigating this important time in their life. They need someone who acknowledges that their education path is unique. They need a partner willing to evolve and not only meet but surpass their expectations. We’re changing. Because students need a better way.
We’re looking for people who are excited to drive this transformation. To break barriers and think of new ways to adapt, help, and create better experiences for students—and for each other.
This is where diverse backgrounds, beliefs, and perspectives matter. It’s where you’re empowered to bring your authentic self to work.
Feeling your best allows you to do your best. Our benefits take care of the whole you—from physical and mental to 

In [134]:
def get_info(url):
    time.sleep(0.05)
    HEADERS = {
    "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    "Accept-Encoding": "gzip",
    "Accept": "text/html",
    "Accept-Language": "en",
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Cookie": 'shk=1hpej3gtsmkkm801; csrf=CoA78HzgzcFPgttGNZvRzeo_poNSlsETD8MeHvzk-DbdqUWA05JfILc3CC404mI8hu453pOdbLSI81M_LNlZ; _ga=GA1.1.1189312297.1710960528; rq=%5B%22q%3Ddata%2Bscientist%26l%3D%26ts%3D1710972560480%22%5D; __cf_bm=aN_K675TYklod50qxgAAOmQdSQ_7ICBzeOqdWIT.2S0-1710983723-1.0.1.1-IRkjgI3bB9lZNXzFJki1qVhF.mCt3a6p.UIIUq.j02XTQBKySe_KNHPu2onGnuiE23UrES17yJE9I9Kj08nfmg; _ga_9GC5K2RCSP=GS1.1.1710984354.4.0.1710984354.0.0.0; OptanonConsent=isGpcEnabled=0&datestamp=Wed+Mar+20+2024+18%3A25%3A54+GMT-0700+(%E5%8C%97%E7%BE%8E%E5%A4%AA%E5%B9%B3%E6%B4%8B%E5%A4%8F%E4%BB%A4%E6%97%B6%E9%97%B4)&version=202308.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=b14705d6-2bab-4a15-9ca9-885fdfdc93c2&interactionCount=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1%2CC0007%3A0&AwaitingReconsent=false'
    }
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code == 200:
        job_soup = BeautifulSoup(response.text, 'html.parser')

        job_details = {
            'job_title':None,'benifits':None,'qualifications':None,
            'full job description':None}

        try:
            job_title = job_soup.find('div',{'class':'chakra-stack css-1iblfv6',})
            job_title = job_title.text
            job_details['job_title'] = job_title
        except:
            pass
        
        try:
            benifites = job_soup.find_all('span',{'data-testid':'viewJobBenefitItem'})
            benifites = [i.text for i in benifites]
            job_details['benifits'] = benifites
        except:
            pass

        try:
            qualifications = job_soup.find_all('span',{'data-testid':'viewJobQualificationItem'})
            qualifications = [i.text for i in qualifications]
            job_details['qualifications'] = qualifications
        except:
            pass

        try:
            full_job_description = job_soup.find('div',{'data-testid':'viewJobBodyJobFullDescriptionContent'})
            full_job_description = full_job_description.text
            job_details['full job description'] = full_job_description
        except:
            pass


    return job_details

In [137]:
all_job_details = {}

In [138]:
i=0
for url in tqdm(all_job_urls[:5000], desc="Fetching job details", unit="job"):
    job_detail = get_info(url)
    all_job_details[i] = job_detail
    i=i+1
print("All job details scraped!")

Fetching job details:  52%|█████▏    | 2605/5000 [37:02<34:02,  1.17job/s]  


UnboundLocalError: cannot access local variable 'job_details' where it is not associated with a value

In [131]:
all_job_details

,job_title,benifits,qualifications,full job description
0,"Manager, Model Validation",None,None,"When you join Sallie Mae, you become a champio..."
1,Data Scientist,None,None,Syntelligent is hiring for a full-time Data Sc...
2,Data Scientist,None,None,"At NSA Data Science is a broad field, and a te..."
3,Data Scientist- 33130,None,None,Description:The Data Scientist will lead proje...
4,Data Analyst / Data Scientist,None,None,"We are small, technology consulting firm assis..."


In [143]:
import json

with open('simplyhired_jobs_detail.json', 'w') as json_file:
    json.dump(all_job_details, json_file)

In [140]:
all_job_details[0]

{'job_title': 'Manager, Model Validation',
 'benifits': ['Paid time off', 'Volunteer time off'],
 'qualifications': ['Financial modeling',
  '5 years',
  'R',
  'Mathematics',
  'Finance',
  "Bachelor's degree",
  'Risk management',
  'Quantitative analysis',
  'SaaS',
  'RMF',
  'Communication skills',
  'Python'],
 'full job description': "When you join Sallie Mae, you become a champion for all students.\nWe’re on a mission to power confidence as students begin their unique journey. To help them plan their higher education, successfully finish, and prepare for life after school. To help them Start smart. Learn big.\nStudents need guidance navigating this important time in their life. They need someone who acknowledges that their education path is unique. They need a partner willing to evolve and not only meet but surpass their expectations. We’re changing. Because students need a better way.\nWe’re looking for people who are excited to drive this transformation. To break barriers and

In [6]:
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

ModuleNotFoundError: No module named 'pyLDAvis.sklearn'

In [ ]:

# Assume 'corpus' contains your preprocessed text data

# Create a vectorizer to convert text data to a bag-of-words matrix
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(corpus)

# Initialize and fit the LDA model
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(X)

# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda_model, X, vectorizer, mds='mmds')
pyLDAvis.save_html(vis, 'lda_visualization.html')  # Save the visualization to an HTML file


In [3]:
import json

with open('simplyhired_jobs_detail.json', 'r', encoding='utf-8') as file:
    all_job_data = file.read()

# 将 JSON 数据转换为字典
all_job_details = json.loads(all_job_data)

In [5]:
all_job_details['0']

{'job_title': 'Manager, Model Validation',
 'benifits': ['Paid time off', 'Volunteer time off'],
 'qualifications': ['Financial modeling',
  '5 years',
  'R',
  'Mathematics',
  'Finance',
  "Bachelor's degree",
  'Risk management',
  'Quantitative analysis',
  'SaaS',
  'RMF',
  'Communication skills',
  'Python'],
 'full job description': "When you join Sallie Mae, you become a champion for all students.\nWe’re on a mission to power confidence as students begin their unique journey. To help them plan their higher education, successfully finish, and prepare for life after school. To help them Start smart. Learn big.\nStudents need guidance navigating this important time in their life. They need someone who acknowledges that their education path is unique. They need a partner willing to evolve and not only meet but surpass their expectations. We’re changing. Because students need a better way.\nWe’re looking for people who are excited to drive this transformation. To break barriers and

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pandas as pd

In [13]:
benefits_texts = []
qualifications_texts = []
for key, value in all_job_details.items():
    benefits_texts=benefits_texts+value['benifits']
    qualifications_texts=qualifications_texts+value['qualifications']

print(len(benefits_texts), len(qualifications_texts))


10147 40966


In [23]:
detail_texts = []
for key, value in all_job_details.items():
    detail_texts.append(value['full job description'])

print(len(detail_texts))

2605


In [15]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim

In [20]:
# 使用 CountVectorizer 对文本进行词频分析
vectorizer_ben = CountVectorizer()
X_ben = vectorizer_ben.fit_transform(benefits_texts)

# 创建 LDA 模型并拟合数据
lda_model_ben = LatentDirichletAllocation(n_components=3, random_state=42)  # 假设选择5个主题
lda_model_ben.fit(X_ben)

texts_tokenized_ben = [text.split() for text in benefits_texts]
dictionary_ben = Dictionary(texts_tokenized_ben)
corpus_ben = [dictionary_ben.doc2bow(text) for text in texts_tokenized_ben]

# 使用 gensim 进行 LDA 模型拟合
lda_gensim_model_ben = LdaModel(corpus=corpus_ben, num_topics=3, id2word=dictionary_ben)

# 可视化 LDA 模型
pyLDAvis.enable_notebook()
panel_ben = pyLDAvis.gensim.prepare(lda_gensim_model_ben, corpus_ben, dictionary_ben)
pyLDAvis.save_html(panel_ben, 'benefits_lda.html')

In [25]:
# 使用 CountVectorizer 对文本进行词频分析
vectorizer_qual = CountVectorizer()
X_qual = vectorizer_qual.fit_transform(qualifications_texts)

# 创建 LDA 模型并拟合数据
lda_model_qual = LatentDirichletAllocation(n_components=5, random_state=42)  # 假设选择5个主题
lda_model_qual.fit(X_qual)

texts_tokenized_qual = [text.split() for text in qualifications_texts]
dictionary_qual = Dictionary(texts_tokenized_qual)
corpus_qual = [dictionary_qual.doc2bow(text) for text in texts_tokenized_qual]

# 使用 gensim 进行 LDA 模型拟合
lda_gensim_model_qual = LdaModel(corpus=corpus_qual, num_topics=5, id2word=dictionary_qual)

# 可视化 LDA 模型
pyLDAvis.enable_notebook()
panel_qual = pyLDAvis.gensim.prepare(lda_gensim_model_qual, corpus_qual, dictionary_qual)
pyLDAvis.save_html(panel_qual, 'qual_lda.html')

In [27]:
detail_texts = list(filter(lambda x: x is not None, detail_texts))

In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# 下载停用词列表
nltk.download('stopwords')
nltk.download('punkt')

# 加载停用词列表
stop_words = set(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/masheng/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [30]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [31]:
import nltk.corpus

In [32]:
stop_words = set(stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/masheng/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [33]:
stopwords = nltk.corpus.stopwords.words("english")
[w for w in detail_texts if w not in stopwords]

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/masheng/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.11/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# 过滤文本中的停用词
filtered_texts = []
for text in detail_texts:
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    filtered_texts.append(' '.join(filtered_text))

In [28]:
# 使用 CountVectorizer 对文本进行词频分析
vectorizer_detail = CountVectorizer()
X_detail = vectorizer_detail.fit_transform(detail_texts)

# 创建 LDA 模型并拟合数据
lda_model_detail = LatentDirichletAllocation(n_components=5, random_state=42)  # 假设选择5个主题
lda_model_detail.fit(X_detail)

texts_tokenized_detail = [text.split() for text in detail_texts]
dictionary_detail = Dictionary(texts_tokenized_detail)
corpus_detail = [dictionary_detail.doc2bow(text) for text in texts_tokenized_detail]

# 使用 gensim 进行 LDA 模型拟合
lda_gensim_model_detail = LdaModel(corpus=corpus_detail, num_topics=5, id2word=dictionary_detail)

# 可视化 LDA 模型
pyLDAvis.enable_notebook()
panel_detail = pyLDAvis.gensim.prepare(lda_gensim_model_detail, corpus_detail, dictionary_detail)
pyLDAvis.save_html(panel_detail, 'detail_lda.html')